In [1]:
#Importing the required packages
import requests
import pandas as pd
import sqlite3
connection=sqlite3.connect(r'C:\Users\VIGNESH V\OneDrive\Desktop\code\project.db')
cursor=connection.cursor()



In [2]:
# Collecting the data by GitHub API base URL
BASE_URL = 'https://api.github.com/search/repositories'

# List of trending topics
topics = [
    'machine learning',
    'data visualization',
    'deep learning',
    'natural language processing',
    'ui/ux',
    'cybersecurity',
    'Blockchain Technology',
    'Data Engineering',
    'Big Data',
    'Test Automation'
]

# Function to fetch repositories for each topic
def fetch_repos(topic):
    url = f"{BASE_URL}?q={topic}&sort=stars&order=desc"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['items']
    else:
        return []

# Store data in a list
data = []
for topic in topics:
    repos = fetch_repos(topic)
    for repo in repos:
        data.append({
            "Topic": topic,  # Added a comma here
            'Repository_Name': repo['name'],
            'Owner': repo['owner']['login'],
            'Description': repo['description'],
            'URL': repo['html_url'],
            'Programming_Language': repo.get('language', 'N/A'),
            'Creation_Date': repo['created_at'],
            'Last_Updated_Date': repo['updated_at'],
            'Number_of_Stars': repo['stargazers_count'],
            'Number_of_Forks': repo['forks_count'],
            'Number_of_Open_Issues': repo['open_issues_count'],
            'License_Type': repo['license']['name'] if repo['license'] else 'No License'
        })

# Convert to DataFrame
df = pd.DataFrame(data)
print(df.head(1))



              Topic Repository_Name       Owner  \
0  machine learning      tensorflow  tensorflow   

                                         Description  \
0  An Open Source Machine Learning Framework for ...   

                                        URL Programming_Language  \
0  https://github.com/tensorflow/tensorflow                  C++   

          Creation_Date     Last_Updated_Date  Number_of_Stars  \
0  2015-11-07T01:19:20Z  2024-10-03T17:58:12Z           185819   

   Number_of_Forks  Number_of_Open_Issues        License_Type  
0            74230                   5026  Apache License 2.0  


In [3]:
#checking the data is having any null values
df.isna().sum()

Topic                     0
Repository_Name           0
Owner                     0
Description               1
URL                       0
Programming_Language     55
Creation_Date             0
Last_Updated_Date         0
Number_of_Stars           0
Number_of_Forks           0
Number_of_Open_Issues     0
License_Type              0
dtype: int64

In [4]:
df["Programming_Language"]=df["Programming_Language"].fillna(df["Programming_Language"].mode()[0])

In [5]:
df=df.dropna()

In [6]:
querry="""CREATE TABLE IF NOT EXISTS repositories (id INTEGER PRIMARY KEY AUTOINCREMENT, Repository_Name VARCHAR(255),
    Owner VARCHAR(255),
    Description TEXT,
    URL VARCHAR(255),
    Programming_Language VARCHAR(100),
    Creation_Date DATETIME,
    Last_Updated_Date DATETIME,
    Number_of_Stars INT,
    Number_of_Forks INT,
    Number_of_Open_Issues INT,
    License_Type VARCHAR(100),Topic VARCHAR(100)
)"""
cursor.execute(querry)

In [7]:
for i, row in df.iterrows():
    cursor.execute('''
        INSERT INTO repositories (Repository_Name, Owner, Description, URL, Programming_Language, Creation_Date, 
                                  Last_Updated_Date, Number_of_Stars, Number_of_Forks, Number_of_Open_Issues, License_Type,Topic)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row['Repository_Name'], row['Owner'], row['Description'], row['URL'], row['Programming_Language'],
          row['Creation_Date'], row['Last_Updated_Date'], row['Number_of_Stars'], row['Number_of_Forks'], 
          row['Number_of_Open_Issues'], row['License_Type'],row["Topic"]))

# Commit the changes to the database
connection.commit()

In [8]:
df_sql=pd.read_sql("select * from repositories",connection)

In [9]:
df_sql

,id,Repository_Name,Owner,Description,URL,Programming_Language,Creation_Date,Last_Updated_Date,Number_of_Stars,Number_of_Forks,Number_of_Open_Issues,License_Type,Topic
0,1,tensorflow,tensorflow,An Open Source Machine Learning Framework for ...,https://github.com/tensorflow/tensorflow,C++,2015-11-07T01:19:20Z,2024-10-03T17:58:12Z,185819,74230,5026,Apache License 2.0,machine learning
1,2,transformers,huggingface,🤗 Transformers: State-of-the-art Machine Learn...,https://github.com/huggingface/transformers,Python,2018-10-29T13:56:00Z,2024-10-03T17:18:22Z,132900,26510,1469,Apache License 2.0,machine learning
2,3,ML-For-Beginners,microsoft,"12 weeks, 26 lessons, 52 quizzes, classic Mach...",https://github.com/microsoft/ML-For-Beginners,HTML,2021-03-03T01:34:05Z,2024-10-03T17:12:57Z,69293,14337,6,MIT License,machine learning
3,4,funNLP,fighting41love,中英文敏感词、语言检测、中外手机/电话归属地/运营商查询、名字推断性别、手机号抽取、身份证抽...,https://github.com/fighting41love/funNLP,Python,2018-08-21T11:20:39Z,2024-10-03T16:03:43Z,68045,14437,28,No License,machine learning
4,5,awesome-machine-learning,josephmisiti,A curated list of awesome Machine Learning fra...,https://github.com/josephmisiti/awesome-machin...,Python,2014-07-15T19:11:19Z,2024-10-03T16:58:38Z,65567,14594,6,Other,machine learning
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,241,dusk,laravel,Laravel Dusk provides simple end-to-end testin...,https://github.com/laravel/dusk,PHP,2016-12-06T03:43:52Z,2024-10-01T14:35:45Z,1871,320,3,MIT License,Test Automation
241,242,Poco,AirtestProject,A cross-engine test automation framework based...,https://github.com/AirtestProject/Poco,Python,2018-01-24T03:24:01Z,2024-10-01T06:31:17Z,1750,316,346,Apache License 2.0,Test Automation
242,243,api_automation_test,githublitao,接口自动化测试平台（老平台移步master_old分支）,https://github.com/githublitao/api_automation_...,JavaScript,2018-01-10T10:35:27Z,2024-10-03T05:14:09Z,1544,841,8,MIT License,Test Automation
243,244,ui-auto-monkey,jonathanpenn,UI AutoMonkey is a simple stress testing scrip...,https://github.com/jonathanpenn/ui-auto-monkey,JavaScript,2012-10-20T18:24:05Z,2024-09-19T02:35:17Z,1466,249,17,MIT License,Test Automation
